In [ ]:
import flappy_bird_gym
import neat
import cv2
import numpy as np
import pickle
import gym, math
import matplotlib.pyplot as plt
import time
import visualise

In [ ]:
import math
import visualise
from importlib import reload
reload(visualise)
env = flappy_bird_gym.make("FlappyBird-v0")
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, 'config-feedforward.txt')
env.seed(10)
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        ob = env.reset()
        net = neat.nn.recurrent.RecurrentNetwork.create(genome, config)

        current_max_fitness = 0
        fitness_current = 0
        counter = 0
        xpos = 0
        xpos_max = 0

        done = False

        while not done:

            # env.render() # uncomment to watch
            # time.sleep(.01)

            nnOutput = np.argmax(net.activate((ob[0], ob[1])))
            ob, reward, done, info = env.step(nnOutput)


            xpos = ob[0]
            ypos = ob[1]

            fitness_current += 1/math.e**abs(ypos)


            if reward > 1:
                fitness_current += 10
                reward += 1

            if fitness_current > current_max_fitness:
                current_max_fitness = fitness_current
                counter = 0
            else:
                counter += 1
            
            if done or counter == 300:
                done = True
            
            genome.fitness = fitness_current

p = neat.Population(config)

p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)
winner = p.run(eval_genomes)
node_names = {-1: 'x_pos', -2: 'y_pos', 0: 'No Jump', 1: 'Jump'}
visualise.draw_net(config, winner, True, node_names=node_names)
visualise.plot_stats(stats, ylog=False, view=True)
visualise.plot_average(stats, ylog=False, view=True)
visualise.plot_species(stats, view=True)

with open('winner2.pkl', 'wb') as output:
    pickle.dump(winner, output, 1)



In [ ]:
scorelist = []
def best_model(genome, slowmode=0):
    env = flappy_bird_gym.make("FlappyBird-v0")
    env.seed(10)
    genome = pickle.load(open('winner.pkl', 'rb'))
    ob = env.reset()
    done = False
    net = neat.nn.recurrent.RecurrentNetwork.create(genome, config)

    while not done:
        
        # if slowmode == 1:
        #     time.sleep(0.01)
        #     # env.render()
        # else:
        #     # env.render()

        nnOutput = np.argmax(net.activate((ob[0], ob[1])))
        
        ob, reward, done, info = env.step(nnOutput)

    # print(info['score'])
    scorelist.append(info['score'])
    return scorelist

genome = pickle.load(open('winner.pkl', 'rb'))
for i in range(100):
    scores = (best_model(genome, 0))

print(scores)

In [ ]:
average_scores = sum(scores) / len(scores)
print(average_scores)